In [ ]:
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, Dropout, Conv2DTranspose
from tensorflow.keras import Model

In [ ]:
from skimage.io import imread, imshow
from skimage.transform import resize

In [ ]:
train_path = "train path"
test_path = "test path"

In [ ]:
from tqdm import tqdm

In [ ]:
#number of images
n = len(os.listdir(train_path))

In [ ]:
train_images_directory = os.listdir(train_path)

In [ ]:
#now we will combine the masks (merge)

In [ ]:
img_width = 128
img_height = 128
img_channels = 3

In [ ]:
x_train = np.zeros((n,img_height, img_width, img_channels), dtype = np.unit8)
y_train = np.zeros((n, img_height, img_width, 1), dtype = np.bool)

In [ ]:
  #getting training images
for i in tqdm(range(n)):
  img_path = train_path + "/" + train_images_directory[i] + "/images/"
  img_name = os.listdir(img_path)[0]
  img = imread(img_path + "/" + img_name)[:,:,:,img_channels]
  img = resize(img, (img_height, img_width), mode = "constant", preserve_range = True)

  x_train[i] = img
  mask_path = train_path + "/" + train_images_directory[i] + "/masks/"
  mask_n = os.listdir(mask_path)
  mask = np.zeros([img_height, img_width, 1], dtype = np.bool)

  for j in range(len(mask_n)):
    mask_img = imread(mask_path + "/" + mask_n[j])
    mask_img = np.expand_dims(resize(mask_img, img_height, img_width),mode = "constant",preserve_range=True, axis = -1)
    mask = np.maximum(mask, mask_img)

  y_train[i] = mask

#you wull now have 1 mask image for each image as we have comboned the mutiple mask images for one single img

In [ ]:
#looking at an image
imshow(x_train[0])
plt.show()

In [ ]:
#mask of above img
imshow(y_train[0])
plt.show()

In [ ]:
#UNet Model

# > segmentation consists of classification and localization

inputs = Input((img_height, img_width, img_channels))
x = tf.keras.layers.Lambda(lambda x : x/255)(inputs)

#downsampling
c1 = Conv2D(16,(3,3), activation = "relu", padding = "same")(x)
c1 = Dropout(0.1)(c1) #adding from out side..not compulsory
c1 = Conv2D(16,(3,3), activation = "relu", padding = "same")(c1)
p1 = MaxPool2D((2,2))(c1)

c2 = Conv2D(32,(3,3), activation = "relu", padding = "same")(p1)
c2 = Dropout(0.1)(c2)
c2 = Conv2D(32,(3,3), activation = "relu", padding = "same")(c2)
p2 = MaxPool2D((2,2))(c2)

c3 = Conv2D(64,(3,3), activation = "relu", padding = "same")(p2)
c3 = Dropout(0.1)(c3)
c3 = Conv2D(64,(3,3), activation = "relu", padding = "same")(c3)
p3 = MaxPool2D((2,2))(c3)

c4 = Conv2D(128,(3,3), activation = "relu", padding = "same")(p3)
c4 = Dropout(0.1)(c4)
c4 = Conv2D(128,(3,3), activation = "relu", padding = "same")(c4)
p4 = MaxPool2D((2,2))(c4)

c5 = Conv2D(256,(3,3), activation = "relu", padding = "same")(p4)
c5 = Dropout(0.1)(c5)
c5 = Conv2D(256,(3,3), activation = "relu", padding = "same")(c5)

#upsampling
u6 = Conv2DTranspose(128,(2,2), strides = (2,2), padding = "same")(c5)
u6 = tf.keras.layers.concatenate([u6,c4])
c6 = Conv2D(128, (3,3), activation. = "relu", padding = "same")(u6)
c6 = Dropout(0.2)(c6)
c6 = Conv2D(128, (3,3), activation. = "relu", padding = "same")(c6)

u7 = Conv2DTranspose(128,(2,2), strides = (2,2), padding = "same")(c6)
u7 = tf.keras.layers.concatenate([u7,c3])
c7 = Conv2D(64, (3,3), activation. = "relu", padding = "same")(u7)
c7 = Dropout(0.2)(c7)
c7 = Conv2D(64, (3,3), activation. = "relu", padding = "same")(c7)

u8 = Conv2DTranspose(32,(2,2), strides = (2,2), padding = "same")(c7)
u8 = tf.keras.layers.concatenate([u8,c2])
c8 = Conv2D(32, (3,3), activation. = "relu", padding = "same")(u8)
c8 = Dropout(0.2)(c8)
c8 = Conv2D(32, (3,3), activation. = "relu", padding = "same")(c8)

u9 = Conv2DTranspose(16,(2,2), strides = (2,2), padding = "same")(c8)
u9 = tf.keras.layers.concatenate([u9,c1])
c9 = Conv2D(16, (3,3), activation. = "relu", padding = "same")(u9)
c9 = Dropout(0.2)(c9)
c9 = Conv2D(16, (3,3), activation. = "relu", padding = "same")(c9)

outputs = Conv2D(1, (1,1), activation = "sigmoid")(c9)

model = Model(inputs = [inputs], outputs = outputs)

In [ ]:
model.summary()

In [ ]:
tf.keras.utils.plot_model(model, to_file = "model.png", show_shapes = True)

In [ ]:
model.compile(optimizer = "adam", loss = "binarycrossentropy", metrics = ["accuracy"])

#including checkpoints (after each iteration(epoch) -- save progress so that it doesnt have to be executed everytime)
checkpoint = tf.keras.callbacks.ModelCheckpoint("model_checkpt.h5", save_best_only = True, verbose = 1)
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience = 2),                   #if after 2 epochs the accuracy does not improve, it will not go further
    tf.keras.callbacks.TensorBoard(log_dir = "logs")
]

In [ ]:
results = model.fit(x_train, y_train, validation_split = 0.1, batch_size = 16, epochs = 10, callbacks = callbacks)

In [ ]:
plt.title("model training plot")
plt.plot(results.history['accuracy'], label = "Train Accuracy")
plt.plot(results.history['val_accuracy'], label = "Validation Accuracy")
plt.legend()
plt.show()

In [ ]:
plt.title("model loss plot")
plt.plot(results.history['loss'], label = "Train Loss")
plt.plot(results.history['val_loss'], label = "Validation Loss")
plt.legend()
plt.show()

In [ ]:
#test model
y_pred_mask = model.predict(x_test)
imshow(x_test[0])
imshow(y_pred_mask[0])


In [ ]:
model.save("unet_model.h5")

In [ ]:
#loading the model when reopening
#make a new file
import tensorflow as tf
from tensorflow.keras.models import load_model

modelll = load_model("unet_model.hf")

img = "imgpath copy paste"

from skimage.io import imread, imshow
from skimage.transform import resize
import numpy as np

img_width = 128
img_height = 128
img_channels = 3
x_test = np.zeros((1,img_height, img_width, img_channels), dtype = np.unit8)
x_test[0] = img

img = imread(img)[:,:,:img_channels]
img = resize(img, (img_height, img_width), mode = "constant", preserve_range = True)

pred_mask = model.predict(img)
imshow(pred_mask.reshape(128,128))
